In [1]:
import glob
import importlib
import json
import sys

import climate_envs
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import torch

In [2]:
BASE_DIR = "/gws/nopw/j04/ai4er/users/pn341/climate-rl"
RECORDS_DIR = f"{BASE_DIR}/records"
ENV_ID = "SimpleClimateBiasCorrection-v0"
EXP_ID = "scbc-v0-optim-L"
STEP_COUNT = 200
NUM_STEPS = 500
SEED = 1

sys.path.append(BASE_DIR)

In [3]:
def make_env(env_id):
    def thunk():
        env = gym.make(env_id)
        return env

    return thunk


def get_actor(algo):
    module_path = f"rl-algos.{algo}.{algo}_actor"
    actor_module = importlib.import_module(module_path)
    Actor = getattr(actor_module, "Actor")
    return Actor


def get_agent(algo):
    module_path = f"rl-algos.{algo}.{algo}_agent"
    actor_module = importlib.import_module(module_path)
    Agent = getattr(actor_module, "Agent")
    return Agent


envs = gym.vector.SyncVectorEnv([make_env(ENV_ID)])

/home/users/p341cam/miniconda3/envs/venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment SimpleClimateBiasCorrection-v0 is out of date. You should consider upgrading to version `v2`.
  logger.deprecation(


Loading NCEP air data ...


<frozen importlib._bootstrap>:241: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


In [4]:
def get_actor_critic_layer_size(pth_fn):
    optim_group = (
        pth_fn.replace("inf_", "")
        .replace("x9_", "")
        .split("/")[-3]
        .split("_")[0]
    )
    if "64" in optim_group:
        actor_layer_size = critic_layer_size = 64
    else:
        with open(
            f"{BASE_DIR}/param_tune/results/{optim_group}/best_results.json",
            "r",
        ) as file:
            opt_params = {
                k: v
                for k, v in json.load(file)[algo].items()
                if k not in {"algo", "episodic_return", "date"}
            }
            for key, value in opt_params.items():
                if key == "actor_critic_layer_size":
                    actor_layer_size = critic_layer_size = value
    return actor_layer_size, critic_layer_size

In [5]:
algo = "ddpg"
record_fn = glob.glob(
    RECORDS_DIR + f"/inf_*{EXP_ID}_*/*{algo}*{SEED}_*/*{STEP_COUNT}.pth"
)[0]
record_steps = torch.load(record_fn)
actor_layer_size, critic_layer_size = get_actor_critic_layer_size(record_fn)

if "inf" not in pattern:
    if algo == "ppo":
        Agent = get_agent(algo)
        agent = Agent(envs, actor_layer_size, critic_layer_size)
        agent.load_state_dict(record_steps["agent"])
    else:
        Actor = get_actor(algo)
        actor = Actor(envs, actor_layer_size)
        actor.load_state_dict(record_steps["actor"])

plt.plot(
    record_steps["global_steps"][-500:],
    record_steps["obs"][-NUM_STEPS:],
    label="state",
)
plt.plot(
    record_steps["global_steps"][-NUM_STEPS:],
    record_steps["actions"][-NUM_STEPS:],
    label="action",
)
plt.xlabel("Steps")
plt.ylabel("Value")
plt.legend();

NameError: name 'pattern' is not defined

In [ ]:
algo = "td3"
record_fn = glob.glob(
    RECORDS_DIR + f"/inf_*{EXP_ID}*/*{algo}*{SEED}_*/*{STEP_COUNT}.pth"
)[0]
record_steps = torch.load(record_fn)
actor_layer_size, critic_layer_size = get_actor_critic_layer_size(record_fn)

plt.plot(
    record_steps["global_steps"][-NUM_STEPS:],
    record_steps["obs"][-NUM_STEPS:],
    label="state",
)
plt.plot(
    record_steps["global_steps"][-NUM_STEPS:],
    record_steps["actions"][-NUM_STEPS:],
    label="action",
)
plt.xlabel("Steps")
plt.ylabel("Value")
plt.legend();